In [1]:
import tensorflow as tf

In [ ]:
random_float = tf.random.uniform(shape=())

In [ ]:
print(random_float)

In [ ]:
random_float

In [ ]:
zero_vector = tf.zeros(shape=(2))

In [ ]:
zero_vector

In [ ]:
print(zero_vector)

In [ ]:
A = tf.constant([[1., 2.], [3., 4.]])

In [ ]:
B = tf.constant([[5., 6.], [7., 8.]])

In [ ]:
print(A.shape)

In [ ]:
print(A.dtype)

In [ ]:
print(A.numpy())

In [ ]:
zero_vector = tf.zeros(shape=(2), dtype=tf.int32)

In [ ]:
zero_vector

In [ ]:
C = tf.add(A, B)

In [ ]:
D = tf.matmul(A, B)

In [ ]:
print(C)

In [ ]:
print(D)

In [ ]:
x = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
    y = tf.square(x)
y_grad = tape.gradient(y, x)
print([y, y_grad])

In [ ]:
print((y, y_grad))

In [ ]:
X = tf.constant([[1., 2.], [3., 4.]])

In [ ]:
y = tf.constant([[1.], [2.]])

In [ ]:
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
    L = 0.5 * tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])
print([L.numpy(), w_grad.numpy(), b_grad.numpy()])

In [ ]:
import numpy as np

In [ ]:
X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

In [ ]:
a, b = 0, 0

num_epoch = 10000
learning_rate = 1e-3
for e in range(num_epoch):
    y_pred = a * X + b
    grad_a, grad_b = (y_pred - y).dot(X), (y_pred - y).sum()
    a, b = a -learning_rate * grad_a, b - learning_rate * grad_b

print(a, b)

In [ ]:
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
for e in range(num_epoch):
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y))
    grads = tape.gradient(loss, variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))
    
print(a, b)

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
    def call(self, input):
        return output

In [ ]:
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])

class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
    
    def call(self, input):
        output = self.dense(input)
        return output

model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.reduce_mean(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)
        self.test_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)
        self.train_label = self.train_label.astype(np.int32)
        self.test_label = self.test_label.astype(np.int32)
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    
    def get_batch(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

In [3]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

In [4]:
import numpy as np
num_epochs = 5
batch_size = 50
learning_rate = 0.001
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [5]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.259120
batch 1: loss 2.276939
batch 2: loss 2.285206
batch 3: loss 2.223507
batch 4: loss 2.086220
batch 5: loss 1.997189
batch 6: loss 1.883638
batch 7: loss 1.787586
batch 8: loss 1.735435
batch 9: loss 1.780396
batch 10: loss 1.715183
batch 11: loss 1.628739
batch 12: loss 1.653841
batch 13: loss 1.345497
batch 14: loss 1.388500
batch 15: loss 1.501500
batch 16: loss 1.404137
batch 17: loss 1.441607
batch 18: loss 1.368984
batch 19: loss 1.192204
batch 20: loss 0.987467
batch 21: loss 1.151904
batch 22: loss 1.252292
batch 23: loss 1.104752
batch 24: loss 0.945899
batch 25: loss 1.102151
batch 26: loss 0.944426
batch 27: loss 0.826030
batch 28: loss 1.199069
batch 29: loss 0.952218
batch 30: loss 0.925899
batch 31: loss 0.788253
batch 32: loss 0.644461
batch 33: loss 0.916108
batch 34: loss 0.746471
batch 35: loss 0.703679
batch 36: loss 0.840046
batch 37: loss 0.968001
batch 38: loss 0.818307
batch 39: loss 0.609879
batch 40: loss 0.917076
batch 41: loss 0.612886
ba

In [15]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    y_true = data_loader.test_label[start_index: end_index]
    #y_true = y_true.reshape(-1,1)
    sparse_categorical_accuracy.update_state(y_true=y_true, y_pred=y_pred)
    
    if batch_index == 1:
        print("y_pred", y_pred.shape)
        print(y_pred)
        print("y_true", y_true.shape)
        print(y_true)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

y_pred (50, 10)
[[0.06355229 0.07759789 0.08952925 0.04277666 0.12865819 0.25102627
  0.0921306  0.09576768 0.07745793 0.08150328]
 [0.06656542 0.07688951 0.08774498 0.04327249 0.1337415  0.24226162
  0.08924366 0.09934657 0.07955554 0.0813788 ]
 [0.06426431 0.07097084 0.08789078 0.04353301 0.1279626  0.24340905
  0.08482601 0.11056635 0.07917313 0.08740396]
 [0.06146048 0.07572088 0.08989947 0.04155692 0.13308713 0.24662761
  0.08957034 0.09990643 0.07683126 0.08533949]
 [0.06174802 0.07170568 0.08872624 0.0420539  0.13461688 0.24541834
  0.08631083 0.10393292 0.07812414 0.08736301]
 [0.06195017 0.07650089 0.08695192 0.0424284  0.1349927  0.24428841
  0.08661191 0.09986589 0.0792086  0.08720113]
 [0.06841341 0.07671553 0.09508575 0.04095772 0.13505627 0.23277201
  0.09123392 0.09914985 0.08024698 0.08036851]
 [0.06132089 0.0744668  0.08917738 0.04120601 0.13532768 0.2463861
  0.08735491 0.10158261 0.07519355 0.08798406]
 [0.05867406 0.07231099 0.09009182 0.04185293 0.14322771 0.24621


InvalidArgumentError: Incompatible shapes: [0] vs. [50] [Op:Equal]

In [16]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output= tf.nn.softmax(x)
        return output

In [ ]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    y_true = data_loader.test_label[start_index: end_index]
    #y_true = y_true.reshape(-1,1)
    sparse_categorical_accuracy.update_state(y_true=y_true, y_pred=y_pred)
    
    if batch_index == 1:
        print("y_pred", y_pred.shape)
        print(y_pred)
        print("y_true", y_true.shape)
        print(y_true)
print("test accuracy: %f" % sparse_categorical_accuracy.result())